In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'TCMD_1'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


df = pd.read_csv('datasets/Tweet_Classification_Moroccan_Dataset/data.csv', encoding='utf-16', engine='python', sep='\t') #, quotechar="'"  , quoting=3

display(len(df))
      
display(df.columns)
display(df[:4])



c = df['Topic'].value_counts()
display(c)

classes = set(df['Topic'].values)
display(classes)

df['Topic'] = df['Topic'].astype('category')
df['label'] = df['Topic'].cat.codes

df = df[['Tweet', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


max_sequence_length = 128



models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',
    'SI2M-Lab/DarijaBERT-mix',
    'SI2M-Lab/DarijaBERT-arabizi'
]


seeds = [0, 1, 42]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
            
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['Tweet'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True)
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 20
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 25, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 25
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2399

Index(['Tweet', 'Topic'], dtype='object')

,Tweet,Topic
0,برا وخيط: أمل بنكيران... مصممة تبتكر الزي المغربي التقليدي بلمسة عالمية,ميديا
1,ربورتاج: قافلة توجيه مُتنقّلة تُطلع تلاميذ العيون على مختلف الآفاق الدراسية,مجتمع
2,ساعة الفطور: طريقة تحضير «وراب» بالخضر وصلصة «تزاتزيكي »,ميديا
3,ربورتاج: مائدة إفطار رمضانية تجمع الديانات الثلاث بالدار البيضاء,ثقافة


Topic
مجتمع     357
رياضة     354
ميديا     353
دولي      348
اقتصاد    344
سياسة     333
ثقافة     310
Name: count, dtype: int64

{'اقتصاد', 'ثقافة', 'دولي', 'رياضة', 'سياسة', 'مجتمع', 'ميديا'}

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT-arabizi and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1919 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
25,1.750700,1.464526,0.485417,0.487555
50,1.176200,1.116076,0.602083,0.604110
75,0.756600,1.165259,0.602083,0.603397
100,0.470300,1.045791,0.677083,0.677819
125,0.277700,1.097797,0.677083,0.680107
150,0.147400,1.160033,0.683333,0.688896
175,0.080700,1.406651,0.664583,0.663667
200,0.071000,1.386584,0.693750,0.697555
225,0.032100,1.411193,0.697917,0.702075
250,0.029400,1.503060,0.691667,0.693671


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.795833,0.795777
3,SI2M-Lab/DarijaBERT-arabizi,0.754167,0.756526
6,SI2M-Lab/DarijaBERT-mix,0.797917,0.796820
12,UBC-NLP/MARBERT,0.833333,0.833643
15,UBC-NLP/MARBERTv2,0.850000,0.849732
18,aubmindlab/bert-base-arabertv02-twitter,0.856250,0.857858
19,qarib/bert-base-qarib,0.845833,0.845078
